In [1]:
from ipywidgets import interact, interactive, fixed, interact_manual, Layout, Output, Button, Checkbox
import ipywidgets as widgets
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import HTML, clear_output, Javascript
from base64 import b64encode
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
%matplotlib inline

##### DEFINE FUNCTIONS #####

def scaling_factor(NA,n1,n2,lam_0):
    z = np.arange(lam_0,100,0.1)
    n2overn1 = np.divide(n2,n1)
    
    if n2overn1 < 1: eps = np.multiply(-1,np.divide(np.divide(lam_0,4),(np.multiply(z,n2))))
    else: eps = np.divide(np.divide(lam_0,4),(np.multiply(z,n2)))
    eps_term = np.multiply(eps, np.subtract(2,eps))    
    
    m = np.emath.sqrt(np.subtract(np.power(n2,2),np.power(n1,2)))
    
    sf_univ = np.multiply(np.divide(n2,n1),
                          np.divide(1-eps+np.divide(m,n1)*np.emath.sqrt(eps_term),
                                    1-np.multiply(np.divide(n2,n1)**2,eps_term)))
    sf_crit = np.divide(n1-np.emath.sqrt(np.power(n1,2)-np.power(NA,2)),
                        n2-np.emath.sqrt(np.power(n2,2)-np.power(NA,2)))
    
    sf = np.zeros(len(z))
    for i in range(len(sf)):
        if n2overn1 < 1: sf[i] = np.max([np.real(sf_univ[i]),np.real(sf_crit)])
        elif n2overn1 > 1:sf[i] = np.min([np.real(sf_univ[i]),np.real(sf_crit)])
        else: sf[i]=1
    return z,sf,sf_crit,n2overn1

def plot_scaling_factor(NA,n1,n2,lam_0,Scaling_theories):
    z,sf,sf_crit,n2overn1=scaling_factor(NA,n1,n2,lam_0)
    plt.figure(figsize=(8,5))

    plt.xlim([0,100])
    plt.xlabel(r'Depth ($\mu$m)')
    plt.ylabel(r'Scaling factor $\zeta$')
    plt.grid(ls=':')
    plt.plot(z,sf,label='Scaling factor',zorder=3000)
    if Scaling_theories == True:
        plt.plot(z,Carlsson(z,n2overn1),label='Carlsson 1991')
        plt.plot(z,Lyakin(z,n2,n1,NA),label=r'Lyakin $et~al.$ 2017')
        plt.plot(z,diel_median(z,n1,n2,NA),label=r'Diel $et~al.$ 2020 (median)')
        if n2 >= NA: 
            plt.plot(z,diel_mean(z,n1,n2,NA),label=r'Diel $et~al.$ 2020 (mean)')
            plt.plot(z,visser(z,n2,n1,NA),label=r'Visser & Oud 1992')
        if n2overn1 >= 1:
            plt.plot(z,stallinga_high(z,n1,n2,[NA]),label=r'Stallinga 2005',ls='--')
    #set ylimits:
    if n2overn1 > 1:   
        plt.ylim([n2overn1*0.95,1.05*np.real(sf_crit)])
    elif n2overn1 < 1: 
        ymin=np.min([0.9*np.real(sf_crit),0.9*float(Lyakin([1],n2,n1,NA))])
        ymax=n2overn1*1.05
        plt.ylim([ymin,ymax])
    else: plt.ylim(0.5,1.5)               
    plt.legend()

def csv_file(x,y):
    # add column names
    string = 'Depth (microns),Scaling Factor\n'
    for i in range(len(x)): #run over all data points and add to file
        string+=(str(x[i])+','+str(y[i])+'\n')
    return string

def trigger_download(text, filename, kind='text/json'):
    # see https://developer.mozilla.org/en-US/docs/Web/HTTP/Basics_of_HTTP/Data_URIs for details
    content_b64 = b64encode(text.encode()).decode()
    data_url = f'data:{kind};charset=utf-8;base64,{content_b64}'
    js_code = f"""
        var a = document.createElement('a');
        a.setAttribute('download', '{filename}');
        a.setAttribute('href', '{data_url}');
        a.click()
    """
    with download_output:
        clear_output()
        display(HTML(f'<script>{js_code}</script>'))

def download_csv(e=None):
    NA = NA_slider.value
    n1 = n1_slider.value
    n2 = n2_slider.value
    lam_0 = lam_slider.value
    z,sf,_,_=scaling_factor(NA,n1,n2,lam_0)
    trigger_download(csv_file(z,sf), 'sf_vs_depth.csv', kind='text/plain')
    
def window_open_button(url):
    with out:
        display(Javascript(f'window.open("{url.tooltip}");'))

##### DEPTH INDEPENDENT THEORIES

def Carlsson(z,n2overn1):
    return np.zeros(len(z)) + n2overn1

def Lyakin(z,n_sample,n_im,NA):
    d = 1
    top = np.add(n_im,np.sqrt(np.subtract(np.power(n_im,2),np.power(NA,2))))
    bottom_1 = np.multiply(4,np.subtract(np.power(n_sample,2),np.power(n_im,2)))
    bottom_2 = np.add(n_im,np.emath.sqrt(np.subtract(np.power(n_im,2),np.power(NA,2))))
    bottom = np.real(np.emath.sqrt(np.add(bottom_1,np.power(bottom_2,2))))
    if bottom == 0: bottom=0.000000000000001
    dz = np.multiply(d,np.divide(top,bottom))
    scaling_factor = np.divide(1,dz)    
    return np.zeros(len(z)) + scaling_factor

def visser(z,n_sample,n_im,NA):
    top = np.tan(np.arcsin(np.divide(NA,n_im)))
    bottom = np.tan(np.arcsin(np.divide(NA,n_sample)))
    sf = np.divide(top,bottom)
    return np.zeros(len(z)) + sf

def diel_mean(z,n_im,n_sample,NA):
    sum=0
    number_of_rays=10000 # paper uses 100, but this is still doable.
    for i in range(number_of_rays):
        k=i+1
        top     =  np.tan(np.arcsin(np.divide((NA*k),(np.multiply(number_of_rays,n_im)))))
        bottom  =  np.tan(np.arcsin(np.divide((NA*k),(np.multiply(number_of_rays,n_sample)))))
        sum +=np.divide(top,bottom)
    return np.zeros(len(z)) + np.divide(sum,number_of_rays)

def diel_median(z,n_im,n_sample,NA):
    top = np.tan(np.arcsin(np.divide(0.5*NA,n_im)))
    bottom = np.tan(np.arcsin(np.divide(0.5*NA,n_sample)))
    return np.zeros(len(z)) + np.divide(top,bottom)

def stallinga_high(z,nn1,nn2,NA):
    if nn1==nn2: 
        return np.ones(len(z))
    alphas=[]
    for i in range(len(NA)):
        alphas.append(-1*(f1f2_av(nn1,nn2,NA[i]) 
                          - f_av(nn1,NA[i]) * f_av(nn2,NA[i])) 
                      / (ff_av(nn1,NA[i]) - f_av(nn1,NA[i])**2))
    d=1
    dz = np.multiply((np.add(alphas,1)),d) #we take delta d as 1
    scaling_factor = np.divide((d-dz),d)
    
    sf = np.divide(-1,alphas)
    return np.zeros(len(z)) + sf

def f_av(nn,NA):
    f=2*(nn**3-(nn**2-NA**2)**(3/2))/(3*NA**2)
    return f

def ff_av(nn,NA):
    ff=nn**2-(NA**2)/2
    return ff

def f1f2_av(nn1,nn2,NA):
    f1f2 = ( (nn1*nn2**3+nn2*nn1**3 - (nn1**2+nn2**2-2*NA**2)*np.sqrt(nn1**2-NA**2)*np.sqrt(nn2**2-NA**2) 
            - ((nn1**2-nn2**2)**2)*np.log( ( np.sqrt(nn1**2-NA**2) - np.sqrt(nn2**2-NA**2) )/ (nn1-nn2) ) )/(4*NA**2) )
    return f1f2

##### START CODE #####
        
print("Scaling factor calculator \n\nAdjust the sliders to set the parameters of the plot\n")    

#set widget parameters:
style = {'description_width': 'initial'}
slider_width = '500px'#'40%'

n1_init=1
n2_init=1.33
NA_init=0.85
lam_init=0.52
#define widgets:
n1_slider = widgets.FloatSlider(min=1,max=1.52,value=n1_init, step=0.01,
                                description='Immersion refractive index $n_1$',
                               style=style,layout = Layout(width=slider_width))
n2_slider = widgets.FloatSlider(min=1,max=1.52,value=n2_init,step=0.01,
                                description='Sample refractive index $n_2$ --',
                               style=style,layout = Layout(width=slider_width))
NA_slider = widgets.FloatSlider(min=0.1, max=1.45,value=NA_init,step=0.05,
                                description='Numerical aperture ----------',
                               style=style,layout = Layout(width=slider_width))
lam_slider = widgets.FloatSlider(min=0.2, max=0.9,value=lam_init,step=0.02,
                                 description=r'Wavelength $\lambda_0$ ($\mu$m) ---------',
                                style=style,layout = Layout(width=slider_width))
theories_checkbox = widgets.Checkbox(False, description='Show depth-independent axial scaling theories',
                                    layout = Layout(width=slider_width))

#make NA slider dependent on n1
def set_NA_range(n1):
    if n1 > 1.45: NA_slider.max =  1.45
    else: NA_slider.max =  n1

def reset_values(e=None):
    NA_slider.value=NA_init
    n1_slider.value=n1_init
    n2_slider.value=n2_init
    lam_slider.value=lam_init
    
        
#h = widgets.interactive(reset_values)        
i = widgets.interactive(set_NA_range, n1=n1_slider)
j = widgets.interactive(plot_scaling_factor, n1=n1_slider,n2=n2_slider,
                        NA=NA_slider,lam_0=lam_slider,Scaling_theories=theories_checkbox)
display(j)

#make buttons
download_output = Output()
display(download_output)
out = Output()
display(out)

btn = Button(description='Download as .csv')
btn2 = Button(description="Source", tooltip='https://github.com/hoogenboom-group/SF')
btn_reset=Button(description='Reset plot')

btn_reset.on_click(reset_values)
btn.on_click(download_csv)
btn2.on_click(window_open_button)

display(btn_reset,btn,btn2)


Scaling factor calculator 

Adjust the sliders to set the parameters of the plot



interactive(children=(FloatSlider(value=0.85, description='Numerical aperture ----------', layout=Layout(width…

Output()

Output()

Button(description='Reset plot', style=ButtonStyle())

Button(description='Download as .csv', style=ButtonStyle())

Button(description='Source', style=ButtonStyle(), tooltip='https://github.com/hoogenboom-group/SF')